# Augment Dataset with Massachusetts roads dataset

## [Dataset kaggle link](https://www.kaggle.com/datasets/insaff/massachusetts-roads-dataset/download?datasetVersionNumber=1)

The dataset is too big (~5 Go) so first we will only take an subset of it

Click on the link to download the whole dataset, unzip it and place the folder ´road_segmentation_ideal´ in the directory ´data/´. Then, run the notebook to take only a cropped subset of the downloaded dataset. 

In [ ]:
import os
from random import sample
import shutil
from utils import *

SUBSET_SIZE = 160
DATA_DIR = '../data/'

In [ ]:
if not os.path.isdir(DATA_DIR + 'road_segmentation_ideal/'):
    print('please download the dataset and place it in the folder data/subset')

In [ ]:
train_in_dst_dir = DATA_DIR +'road_segmentation_ideal_subset/training/input/'
if not os.path.isdir(train_in_dst_dir):
    os.makedirs(train_in_dst_dir)

train_out_dst_dir = DATA_DIR + 'road_segmentation_ideal_subset/training/output/'
if not os.path.isdir(train_out_dst_dir):
    os.makedirs(train_out_dst_dir)

outputs = os.listdir(DATA_DIR + 'road_segmentation_ideal/training/output/')
random_elements = sample(outputs, SUBSET_SIZE)
for name in random_elements :

    train_in_src =  DATA_DIR +f'road_segmentation_ideal/training/input/{name}'
    train_in_dst = train_in_dst_dir + f'{name}'
    shutil.copy(train_in_src, train_in_dst)

    train_out_src =  DATA_DIR +f'road_segmentation_ideal/training/output/{name}'
    train_out_dst = train_out_dst_dir + f'{name}'
    shutil.copy(train_out_src, train_out_dst)

Copy test directory

In [ ]:
from distutils.dir_util import copy_tree

test_src_dir = DATA_DIR + 'road_segmentation_ideal/testing/'
test_dst_dir = DATA_DIR + 'road_segmentation_ideal_subset/testing/'

test_dst_in_dir = test_dst_dir + 'input'
if not os.path.isdir(test_dst_in_dir):
    os.makedirs(test_dst_in_dir)

test_dst_out_dir = test_dst_dir + 'output'
if not os.path.isdir(test_dst_out_dir):
    os.makedirs(test_dst_out_dir)

copy_tree(test_src_dir, test_dst_dir)

### Explore the data

In [ ]:
# images = load_all_from_path(train_in_dst_dir)
# masks = load_all_from_path(train_out_dst_dir)

In [ ]:
# show_first_n(images, masks, 3)

In [ ]:
# images[0].shape

### We can see that images from Massachusetts dataset are (1500x1500) so we will randomly crop them so they are (400x400)

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from PIL import Image
CROP_SIZE = 400

In [ ]:
cropped_directory = DATA_DIR + 'road_segmentation_ideal_subset_cropped/'
if not os.path.isdir(cropped_directory):
    os.mkdir(cropped_directory)

Crop training images 

In [ ]:
train_crop_out_dst_dir = cropped_directory + 'training/output/'
if not os.path.isdir(train_crop_out_dst_dir):
    os.makedirs(train_crop_out_dst_dir)

train_crop_in_dst_dir = cropped_directory + 'training/input/'
if not os.path.isdir(train_crop_in_dst_dir):
    os.makedirs(train_crop_in_dst_dir)
    
for name in os.listdir(DATA_DIR + 'road_segmentation_ideal_subset/training/output/'):
    image = Image.open(train_in_dst_dir + f'{name}')
    mask = Image.open(train_out_dst_dir + f'{name}')
    transform = transforms.RandomCrop(CROP_SIZE)
    i, j, h, w = transform.get_params(image, output_size=(CROP_SIZE, CROP_SIZE))
    image_crop = TF.crop(image, i, j, h, w)
    mask_crop = TF.crop(mask, i, j, h, w)
    image_crop.save(train_crop_in_dst_dir + f'{name}')
    mask_crop.save(train_crop_out_dst_dir + f'{name}')

Crop testing images

In [ ]:
test_crop_in_dir = cropped_directory + 'testing/input/'
if not os.path.isdir(test_crop_in_dir):
    os.makedirs(test_crop_in_dir)

test_crop_out_dir = cropped_directory + 'testing/output/'
if not os.path.isdir(test_crop_out_dir):
    os.makedirs(test_crop_out_dir)

test_out_dst_dir = DATA_DIR + 'road_segmentation_ideal_subset/testing/output/'
test_in_dst_dir = DATA_DIR + 'road_segmentation_ideal_subset/testing/input/'
for name in os.listdir(test_out_dst_dir):
    image = Image.open(test_in_dst_dir + f'{name}')
    mask = Image.open(test_out_dst_dir + f'{name}')
    transform = transforms.RandomCrop(CROP_SIZE)
    i, j, h, w = transform.get_params(image, output_size=(CROP_SIZE, CROP_SIZE))
    image_crop = TF.crop(image, i, j, h, w)
    mask_crop = TF.crop(mask, i, j, h, w)
    image_crop.save(test_crop_in_dir + f'{name}')
    mask_crop.save(test_crop_out_dir + f'{name}')

Once we have create the cropped subset, we can delete the subset directory 

In [ ]:
subset_dir = DATA_DIR + 'road_segmentation_ideal_subset/'
if os.path.isdir(subset_dir):
    shutil.rmtree(subset_dir)

Now, we split the cropped train directory into training and validation
We do so by taking at random 10 elements of the training set and putting them into validation set

In [ ]:
VALIDATION_SIZE = int(SUBSET_SIZE * 30 / 100)

val_dir = DATA_DIR + 'road_segmentation_ideal_subset_cropped/validation/'
if not os.path.isdir(val_dir):
    os.mkdir(val_dir)

val_dir_in = val_dir + 'input/'
if not os.path.isdir(val_dir_in):
    os.mkdir(val_dir_in)

val_dir_out = val_dir + 'output/'
if not os.path.isdir(val_dir_out):
    os.mkdir(val_dir_out)

In [ ]:
train_in_dir = DATA_DIR + 'road_segmentation_ideal_subset_cropped/training/input/'
train_out_dir = DATA_DIR + 'road_segmentation_ideal_subset_cropped/training/output/'

train_elements = os.listdir(train_in_dir)
rdm_val_elements = sample(train_elements, VALIDATION_SIZE)
for val_el in rdm_val_elements:
    shutil.move(train_in_dir + val_el, val_dir_in + val_el)
    shutil.move(train_out_dir + val_el, val_dir_out + val_el)